In [14]:
import spacy
import os
import pandas as pd
from spacy.matcher import Matcher
from dashboardContent import func

nlp = spacy.load("en_core_web_sm")
from sklearn.preprocessing import MinMaxScaler
import texthero as hero

In [2]:
Condition = ['depending', 'necessary', 'appropriate',
                 'inappropriate', 'as needed', 'as applicable',
                 'otherwise reasonably', 'sometimes',
                 'from time to time']
Generalization = ['generally', 'mostly', 'widely',
                  'general', 'commonly',
                  'usually', 'normally', 'typically',
                  'largely', 'often', 'primarily',
                  'among other things']
Modality = ['may', 'might', 'can', 'could', 'would',
            'likely', 'possible', 'possibly']
Numeric_quantifier = ['anyone', 'certain', 'everyone',
                      'numerous', 'some', 'most', 'few',
                      'much', 'many', 'various',
                      'including but not limited to']

In [3]:
Path = "Privacy_Policies/"
filelist = os.listdir(Path)
txt_files = []
file_names = []
for i in filelist:
    if i.endswith(".txt"):
        file_names.append(i)
        with open(Path + i, 'r') as f:
            txt_files.append(f.read())

## Bradley-Terry Coeff

In [4]:
bt_coef ={
    "CN": 1.619,
    "C": 1.783,
    "CM": 1.864,
    "CMN": 2.125,
    "CG": 2.345,
    "CGN": 2.443,
    "MN": 2.569,
    "N": 2.710,
    "M": 2.865,
    "CGMN": 2.899,
    "CGM": 2.968,
    "GN": 3.281,
    "GMN": 3.506,
    "G": 3.550,
    "GM": 4.045
}

## Policy String

In [5]:
Policies = pd.DataFrame([file_names, txt_files]).T
Policies.columns = ["File_name", "text_content"]

In [6]:
Policies

,File_name,text_content
0,InternetSpeedMeter.txt,Internet Speed MeterSearch this siteInternet S...
1,VideoEditor&VideoMaker-InShot.txt,WelcomePrivacy Policy - InShot Inc.Protecting ...
2,Josh-SnackonShortVideoswithTopIndianApp.txt,Privacy PolicyContent PolicyUser AgreementTerm...
3,Telegram.txt,Telegram Privacy Policy EnglishBahasa Indonesi...
4,Dailyhunt-100%IndianAppforNews&Videos.txt,Privacy - DailyhuntAbout UsCareersContact UsAb...
...,...,...
71,"AirtelThanks-Recharge,BillPay,Bank,LiveTV.txt",We at Bharti Airtel Limited and Bharti Hexacom...
72,FLStudioMobile.txt,MOBILE APP PRIVACY | FL Studio FL StudioOvervi...
73,QuickHealTotalSecurity.txt,PRIVACY POLICY- QUICK HEAL MOBILE DEVICE SECUR...
74,DroidCamX.txt,Dev47AppsDroidCam Privacy Policy and Terms of ...


In [9]:
from texthero import preprocessing
my_pipe = [preprocessing.fillna,
          preprocessing.remove_diacritics,
          preprocessing.remove_punctuation,
          preprocessing.remove_whitespace]
Policies['clean-text'] = hero.clean(Policies['text_content'], my_pipe)

Policies['clean-text'] = [n.replace(')', '') for n in Policies['clean-text']]
Policies['clean-text'] = [n.replace('(', '') for n in Policies['clean-text']]
Policies['clean-text'] = [n.replace('{', '') for n in Policies['clean-text']]
Policies['clean-text'] = [n.replace('}', '') for n in Policies['clean-text']]

In [11]:
Policies['File_name'] = Policies['File_name'].map(lambda x: str(x)[:-4])
Policies['File_name']= Policies['File_name'].str.replace("-"," ")
Policies['File_name']= Policies['File_name'].str.replace(","," ")
res = []
for ele in Policies['File_name']:
    temp = [[]]
    for char in ele:

        # checking for upper case character
        if char.isupper():
            temp.append([])

        # appending character at latest list
        temp[-1].append(char)

    # joining lists after adding space
    res.append(' '.join(''.join(ele) for ele in temp))
Policies["File_name"] = res

Policies.head()

,File_name,text_content,clean-text
0,Internet Speed Meter,Internet Speed MeterSearch this siteInternet S...,Internet Speed MeterSearch this siteInternet S...
1,Video Editor& Video Maker In Shot,WelcomePrivacy Policy - InShot Inc.Protecting ...,WelcomePrivacy Policy InShot Inc Protecting yo...
2,Josh Snackon Short Videoswith Top Indian App,Privacy PolicyContent PolicyUser AgreementTerm...,Privacy PolicyContent PolicyUser AgreementTerm...
3,Telegram,Telegram Privacy Policy EnglishBahasa Indonesi...,Telegram Privacy Policy EnglishBahasa Indonesi...
4,Dailyhunt 100% Indian Appfor News& Videos,Privacy - DailyhuntAbout UsCareersContact UsAb...,Privacy DailyhuntAbout UsCareersContact UsAbou...


In [16]:
# Getting sentences
Policies["Sentences"] = Policies["clean-text"].apply(lambda x: [sent.text for sent in nlp(x).sents])

# Getting lenth of sentences
Policies['Length'] = Policies['Sentences'].apply(lambda x: len(x))
Policies

KeyboardInterrupt: 

## Generating Outputs

In [15]:
coefs = []
vagueness_cats = { "cat":[], "score":[] }
for i in Policies.iterrows():
    file_name = i[1][0][:-4]
    file_path = "new_results/" + file_name + ".csv"
    temp_df = func.make_df(i[1][1])
    if "may" in temp_df:
        temp_df.to_csv(file_path)
#         coefs.append(temp_df["BT Coeff"].mean())
#         coefs.append(temp_df['Amb_Terms'])
#         for num, i in enumerate(temp_df["BT Coeff"]):
#             vagueness_cats["score"].append(i)
#             vagueness_cats["cat"].append(temp_df["Category"][num])
#     else:
#         coefs.append("NA")
# Policies["vague_score"] = coefs

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'N', 2.71, ['anyone'], ['This page is used to inform visitors regarding our policies with the collection, use, and disclosure of Personal Information if anyone decided to use our Service.']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'N', 2.71, ['anyone'], ['This page is used to inform visitors regarding our policies with the collection, use, and disclosure of Personal Information if anyone decided to use our Service.']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'N', 2.71, ['anyone'], ['We will not use or share your information with anyone except as described in this Privacy Policy.']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'N', 2.7

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 'MN', 2.569, ['may', 'including but not limited to'], ['Personally identifiable Information collected may include Your name, image, mailing address, email address, phone number and demographic information such as gender, nationality, postcode and other personal information including but not limited to date, time or place of birth, location, interests, preferences, likes and dislikes, and usage information.']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 'N', 2.71, ['certain', 'including but not limited to'], ['Verse collects Personally Identifiable Information only if You voluntarily submit such information to us or provide access to such information by connecting to social networking sites through the Platform or Services (for example when You use the facilities or Services connected with th

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['Your email address may be used to send user information, administrative information, changes in account settings and any changes to the Platform/Services or updates on new policies of Verse.']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['Apart from this if You choose to opt-in to our mailing list, You will receive periodic emails that may relate to company news, related product or service information, etc.']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['Apart from this if You choose to opt-in to our mailing list, You will receive periodic emails that may relate to company news, related product or service information, etc.']]
[0, 0, 0, 0, 0, 0, 0, 0,

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['Verse and/or its service providers may use advertising cookies to deliver ads that are more relevant to You and Your interests unless You opt out of such advertisements.6.3.']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['Verse and/or its service providers may use advertising cookies to deliver ads that are more relevant to You and Your interests unless You opt out of such advertisements.6.3.']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['You may choose to disable cookies by turning off cookie feature on the web browser.']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['Table of ContentsThis Privacy Policy explains the following:the legal basis for processing your personal data;what personal data we may collect from you;how']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['Table of ContentsThis Privacy Policy explains the following:the legal basis for processing your personal data;what personal data we may collect from you;how']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['we keep your personal data safe;what we may use your personal data']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['we keep your personal data safe;wh

...

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['you may occasionally provide to us via Internet (the Google Play Platform).Generally, we do not collect any PII from you when you download our Android applications.']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['you may occasionally provide to us via Internet (the Google Play Platform).Generally, we do not collect any PII from you when you download our Android applications.']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'G', 3.55, ['commonly'], ['To be specific, we do not require the consumers to get registered before downloading the applicationCookiesCookies are files with a small amount of data that are commonly used as anonymous unique identifiers.']]
[0, 0, 0, 0, 0, 

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['We may collect your Personal Information when you use our services or websites or otherwise interact with us during the course of our relationship.']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['The Personal Information may be collected by Airtel through the Customer Application Form (CAF) or through its authorized third parties.']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['The Personal Information may be collected by Airtel through the Customer Application Form (CAF) or through its authorized third parties.']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['ma

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may', 'may'], ['Airtel may need to request specific information from you to help us confirm your identity and ensure your right to access your personal data (or to exercise other rights you may have under applicable law).']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may', 'may'], ['Airtel may need to request specific information from you to help us confirm your identity and ensure your right to access your personal data (or to exercise other rights you may have under applicable law).']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['We may also contact you to ask you for further information in relation to your request to speed up our response.']]
[0, 0, 0, 0, 0, 0, 0

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['Storage of Your InformationThe information we collect from you may be stored on servers in India and anywhere Quick Heal or our vendors operate.']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['Links to Other WebsitesOur App/websites may contain links to other Apps/websites, which are owned or operated by other companies.']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['Links to Other WebsitesOur App/websites may contain links to other Apps/websites, which are owned or operated by other companies.']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['If you cho

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 'N', 2.71, ['including but not limited to'], ['WHETHER IN AN ACTION IN CONTRACT, TORT (INCLUDING BUT NOT LIMITED TO NEGLIGENCE) OR OTHERWISE,ARISING OUT OF OR']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['This document is not intended to alter the terms or conditions of any other license agreement you may have with Dev47Apps, its subsidiaries or affiliates,and to the extent of any conflict, the terms of your license agreement will prevail.-[1]']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['This document is not intended to alter the terms or conditions of any other license agreement you may have with Dev47Apps, its subsidiaries or affiliates,and to the extent of any conf

In [18]:
Policies

,File_name,text_content
0,InternetSpeedMeter.txt,Internet Speed MeterSearch this siteInternet S...
1,VideoEditor&VideoMaker-InShot.txt,WelcomePrivacy Policy - InShot Inc.Protecting ...
2,Josh-SnackonShortVideoswithTopIndianApp.txt,Privacy PolicyContent PolicyUser AgreementTerm...
3,Telegram.txt,Telegram Privacy Policy EnglishBahasa Indonesi...
4,Dailyhunt-100%IndianAppforNews&Videos.txt,Privacy - DailyhuntAbout UsCareersContact UsAb...
...,...,...
71,"AirtelThanks-Recharge,BillPay,Bank,LiveTV.txt",We at Bharti Airtel Limited and Bharti Hexacom...
72,FLStudioMobile.txt,MOBILE APP PRIVACY | FL Studio FL StudioOvervi...
73,QuickHealTotalSecurity.txt,PRIVACY POLICY- QUICK HEAL MOBILE DEVICE SECUR...
74,DroidCamX.txt,Dev47AppsDroidCam Privacy Policy and Terms of ...


In [19]:
# !zip -r result.zip output

# Manual Testing

In [20]:
make_df("i may be hahahehe")

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['i may be hahahehe']]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'M', 2.865, ['may'], ['i may be hahahehe']]


,depending,necessary,appropriate,inappropriate,as needed,as applicable,otherwise reasonably,sometimes,from time to time,generally,...,most,few,much,many,various,including but not limited to,Category,BT Coeff,Amb_Terms,Amb_Phrase
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,M,2.865,[may],[i may be hahahehe]


In [ ]:
Policies.to_csv("Policies.csv")

In [21]:
for num, i in enumerate(Policies["File_name"]):
  print(i,": ", Policies["vague_score"][num])

Policies["vague_score"].describe()

KeyError: 'vague_score'

In [ ]:
vagueness_cats = pd.DataFrame(vagueness_cats)
vagueness_cats = vagueness_cats.groupby(["cat"]).count()
vagueness_cats.columns = ["Count"]
vagueness_cats["Count"] /= sum(vagueness_cats["Count"])
vagueness_cats.columns = ["Probablity"]

In [ ]:
plot_data = pd.DataFrame([bt_coef]).T
plot_data.columns = ["bt_coef"]
temp = []
for i in bt_coef:
    try:
        temp.append(vagueness_cats.T[i][0])
    except KeyError :
        temp.append(0)
plot_data["Probablity"] = temp

In [ ]:
plot_data

,bt_coef,Probablity
CN,1.619,0.007133
C,1.783,0.057405
CM,1.864,0.052649
CMN,2.125,0.004076
CG,2.345,0.000679
CGN,2.443,0.000340
MN,2.569,0.094090
N,2.710,0.112772
M,2.865,0.634511
CGMN,2.899,0.000000


In [22]:
plot_data.to_csv("coef_prob_plot.csv")

NameError: name 'plot_data' is not defined

In [19]:
a = pd.read_csv("new_results/ A J I O Online Shopping  Handpicked Curated Fas.csv")

In [21]:
a.iloc[0,]

"['various']"